In [1]:
!pip install yfinance
!pip install pykalman
!pip install TA-Lib-binary
!pip install fracdiff
!pip install catboost



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import talib as ta
from pykalman import KalmanFilter
from fracdiff.sklearn import Fracdiff

In [3]:
train_start='1928-01-01'
train_end='2008-12-31'
tickers = ['^GSPC']

In [4]:
train_df = yf.download(tickers,start=train_start,end=train_end,parse_dates=True,interval = "1wk")

[*********************100%***********************]  1 of 1 completed


In [5]:
def data_engineering(df: pd.DataFrame) -> pd.DataFrame:
    f = Fracdiff(0.3,window=3)
    a = f.fit_transform(df)
    statdf=pd.DataFrame(data = a, 
                        index = df.index, 
                        columns = df.columns)
    pred=statdf['Adj Close'].shift(-1)
    statdf['pred']=pred
    statdf.dropna(inplace=True)
    atr= ta.ATR(statdf['High'], statdf['Low'], statdf['Close'], timeperiod=3)
    rsi = ta.RSI(statdf['Close'], timeperiod=3)
    obv = ta.OBV(statdf['Close'], statdf['Volume'])
    wcp = ta.WCLPRICE(statdf['High'], statdf['Low'], statdf['Close'])
    trend = ta.HT_TRENDMODE(statdf['Close'])
    kf = KalmanFilter(transition_matrices = [1],
                  observation_matrices = [1],
                  initial_state_mean = statdf['Close'].values[0],
                  initial_state_covariance = 1,
                  observation_covariance=1,
                  transition_covariance=.01)
    state_means,_ = kf.filter(statdf[['Close']].values)
    kalf = state_means.flatten()
    statdf['atr']=atr
    statdf['rsi']=rsi
    statdf['obv']=obv
    statdf['wcp']=wcp
    statdf['trend']=trend
    statdf['kalf']=kalf
    statdf['pred_change']=statdf['pred'].pct_change()
    statdf.dropna(inplace=True)
    mask_posbuy = (statdf.pred_change >= 0.00) 
    signalpredbuy= np.where(mask_posbuy, 1, 0)
    mask_possell = (statdf.pred_change < 0.00) 
    signalpredsell= np.where(mask_possell, -1, 0)
    statdf['signal']=signalpredbuy+signalpredsell
    return statdf

In [6]:
train = data_engineering(train_df)

In [7]:
train

,Open,High,Low,Close,Adj Close,Volume,pred,atr,rsi,obv,wcp,trend,kalf,pred_change,signal
Date,,,,,,,,,,,,,,,
1928-01-22,10.615499,10.620099,10.520250,10.600101,10.600101,0.000000e+00,10.257600,2.637716,3.287163,0.000000e+00,10.585138,0,13.617763,-0.032311,-1
1928-01-29,10.382550,10.481599,10.331699,10.257600,10.257600,0.000000e+00,10.462551,1.847945,3.077908,0.000000e+00,10.332124,0,13.008743,0.019980,1
1928-02-05,10.350801,10.401450,10.380401,10.462551,10.462551,0.000000e+00,9.951001,1.279913,8.316639,0.000000e+00,10.426738,0,12.599968,-0.048893,-1
1928-02-12,10.368550,10.323851,9.981001,9.951001,9.951001,0.000000e+00,10.226300,1.013792,6.916910,0.000000e+00,10.051713,0,12.214023,0.027665,1
1928-02-19,9.885751,10.105250,10.006800,10.226300,10.226300,0.000000e+00,10.356799,0.727278,18.050844,0.000000e+00,10.141163,0,11.946236,0.012761,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-11-23,440.872268,531.608255,492.931556,564.535534,564.535534,9.290910e+09,523.194857,87.457626,61.090798,3.171202e+10,538.402720,1,622.150046,-0.073230,-1
2008-11-30,556.560834,527.094538,497.522897,523.194857,523.194857,1.960163e+10,522.803779,80.642630,49.135717,1.211040e+10,517.751787,1,612.736942,-0.000747,-1
2008-12-07,532.001025,557.880762,522.516974,522.803779,522.803779,1.806789e+10,531.973660,65.549683,48.999652,-5.957490e+09,531.501323,1,604.182057,0.017540,1


In [8]:
X_train = train[[ 'Open', 'High', 'Low', 'Adj Close', 'Volume','atr','rsi', 'obv', 'wcp', 'trend', 'kalf']]
y_train = train['signal']

In [9]:
from sklearn.ensemble import AdaBoostClassifier
model1=AdaBoostClassifier(random_state=6, n_estimators=45,learning_rate=0.025)
model1.fit(X_train,y_train)

AdaBoostClassifier(learning_rate=0.025, n_estimators=45, random_state=6)

In [10]:
from catboost import CatBoostClassifier
model2 = CatBoostClassifier(iterations=100,learning_rate=0.025,random_state=6,verbose=2,max_depth=5)
model2.fit(X_train,y_train)

0:	learn: 0.6920932	total: 52.9ms	remaining: 5.24s
2:	learn: 0.6902703	total: 64.5ms	remaining: 2.08s
4:	learn: 0.6886529	total: 69.3ms	remaining: 1.32s
6:	learn: 0.6869735	total: 72.8ms	remaining: 967ms
8:	learn: 0.6854219	total: 76.2ms	remaining: 771ms
10:	learn: 0.6840517	total: 79.4ms	remaining: 642ms
12:	learn: 0.6828323	total: 82.8ms	remaining: 554ms
14:	learn: 0.6817043	total: 86.3ms	remaining: 489ms
16:	learn: 0.6805773	total: 90ms	remaining: 439ms
18:	learn: 0.6794769	total: 94.3ms	remaining: 402ms
20:	learn: 0.6784824	total: 99.7ms	remaining: 375ms
22:	learn: 0.6776210	total: 105ms	remaining: 353ms
24:	learn: 0.6767405	total: 110ms	remaining: 330ms
26:	learn: 0.6759452	total: 113ms	remaining: 306ms
28:	learn: 0.6751500	total: 117ms	remaining: 286ms
30:	learn: 0.6743861	total: 120ms	remaining: 267ms
32:	learn: 0.6738775	total: 124ms	remaining: 251ms
34:	learn: 0.6732600	total: 127ms	remaining: 236ms
36:	learn: 0.6727477	total: 131ms	remaining: 223ms
38:	learn: 0.6721544	total:

In [11]:
test_start='2009-01-01'
test_end='2021-12-31'
test_df = yf.download(tickers,start=test_start,end=test_end,parse_dates=True,interval = "1wk")

[*********************100%***********************]  1 of 1 completed


In [12]:
test = data_engineering(test_df)

In [13]:
X_test = test[[ 'Open', 'High', 'Low', 'Adj Close', 'Volume', 'atr',
       'rsi', 'obv', 'wcp', 'trend', 'kalf']]
y_test = test['signal']

In [14]:
y_pred1 = model1.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

          -1       0.57      0.67      0.62       314
           1       0.66      0.56      0.61       361

    accuracy                           0.61       675
   macro avg       0.62      0.62      0.61       675
weighted avg       0.62      0.61      0.61       675



In [15]:
y_pred2 = model2.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

          -1       0.56      0.71      0.62       314
           1       0.67      0.52      0.58       361

    accuracy                           0.60       675
   macro avg       0.61      0.61      0.60       675
weighted avg       0.62      0.60      0.60       675



In [16]:
signal_strength = y_pred1+y_pred2

In [17]:
signal_strength

array([ 2,  2,  2,  2,  2,  2,  2, -2, -2,  2,  2, -2,  2, -2, -2,  2,  2,
        2, -2, -2,  2,  2,  0,  2, -2, -2, -2, -2, -2,  2, -2,  2,  0, -2,
        2,  2,  2, -2,  2,  2,  2, -2, -2,  0,  2,  2,  0, -2, -2, -2, -2,
        2,  2,  2,  2,  0,  0, -2, -2, -2, -2, -2, -2, -2, -2,  2,  2,  2,
        2,  2,  2,  2, -2,  2,  2,  2, -2,  2, -2, -2,  2,  2,  2,  2,  0,
       -2, -2, -2, -2, -2, -2, -2, -2, -2,  2,  2, -2, -2, -2, -2, -2, -2,
       -2,  2, -2, -2, -2, -2,  2,  2, -2,  2,  0, -2, -2,  2,  0, -2,  0,
        2,  2,  2,  2,  2,  2, -2, -2, -2,  2,  0,  2,  2,  2,  0,  2, -2,
        2,  2,  2,  2,  2, -2, -2, -2, -2,  2, -2,  2, -2, -2,  2,  2,  2,
       -2, -2, -2, -2, -2, -2,  2, -2, -2,  2, -2, -2, -2,  2,  2,  2, -2,
       -2,  2,  2,  2,  2,  2,  2, -2,  2, -2,  2, -2,  2, -2, -2, -2, -2,
        2,  2, -2, -2,  2, -2,  2, -2,  2,  2,  2,  2,  0, -2,  2, -2, -2,
        2, -2, -2, -2, -2, -2, -2, -2,  2, -2, -2, -2, -2, -2,  2, -2,  2,
       -2, -2, -2, -2, -2

In [18]:
signal=signal_strength.squeeze().tolist()

In [19]:
def signal_type(list) -> list:
  for i in range(len(signal)):
    
      if signal[i] == 2:
          signal[i] = 'strong_buy'
  
      if signal[i] == -2:
          signal[i] = 'strong_sell'
    
      if signal[i] == 0:
          signal[i] = 'do_nothing'

      

  return signal
  


In [20]:
signal_type(signal)

['strong_buy',
 'strong_buy',
 'strong_buy',
 'strong_buy',
 'strong_buy',
 'strong_buy',
 'strong_buy',
 'strong_sell',
 'strong_sell',
 'strong_buy',
 'strong_buy',
 'strong_sell',
 'strong_buy',
 'strong_sell',
 'strong_sell',
 'strong_buy',
 'strong_buy',
 'strong_buy',
 'strong_sell',
 'strong_sell',
 'strong_buy',
 'strong_buy',
 'do_nothing',
 'strong_buy',
 'strong_sell',
 'strong_sell',
 'strong_sell',
 'strong_sell',
 'strong_sell',
 'strong_buy',
 'strong_sell',
 'strong_buy',
 'do_nothing',
 'strong_sell',
 'strong_buy',
 'strong_buy',
 'strong_buy',
 'strong_sell',
 'strong_buy',
 'strong_buy',
 'strong_buy',
 'strong_sell',
 'strong_sell',
 'do_nothing',
 'strong_buy',
 'strong_buy',
 'do_nothing',
 'strong_sell',
 'strong_sell',
 'strong_sell',
 'strong_sell',
 'strong_buy',
 'strong_buy',
 'strong_buy',
 'strong_buy',
 'do_nothing',
 'do_nothing',
 'strong_sell',
 'strong_sell',
 'strong_sell',
 'strong_sell',
 'strong_sell',
 'strong_sell',
 'strong_sell',
 'strong_sel

In [22]:
import pickle 
pickle.dump(model1,open('model1.pkl','wb'))
pickle.dump(model2,open('model2.pkl','wb'))
